In [1]:
from happytransformer import HappyTextToText

# t5 is a text-to-text model: given text it generates text based on the input
happy_tt = HappyTextToText("T5", "t5-base") # model type and model name

Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 998kB/s]
Downloading: 100%|██████████| 850M/850M [01:33<00:00, 9.55MB/s] 
Downloading: 100%|██████████| 773k/773k [00:02<00:00, 371kB/s]  
Downloading: 100%|██████████| 1.32M/1.32M [00:02<00:00, 496kB/s] 
05/05/2022 10:58:59 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [2]:
from datasets import load_dataset

train_dataset = load_dataset("jfleg", split='validation[:]')
eval_dataset = load_dataset("jfleg", split='test[:]')

Extracting data files: 100%|██████████| 5/5 [00:00<00:00, 1683.38it/s]


Dataset jfleg downloaded and prepared to /Users/yevgeniy_simonov/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b. Subsequent calls will reuse this data.


In [3]:
# examine dataset
for case in train_dataset["corrections"][:2]:
    print(case)
    print(case[0])
    print("-----------------------------------------------------")

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
-----------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
-----------------------------------------------------


In [33]:
# data processing
import csv
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                if input_text and correction:
                    writer.writerow([input_text, correction])

In [34]:
generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [35]:

from happytransformer import TTTrainArgs

args = TTTrainArgs(batch_size=8)
happy_tt.train("train.csv", args=args)

05/05/2022 11:44:51 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 416.31it/s]


Dataset csv downloaded and prepared to /Users/yevgeniy_simonov/.cache/huggingface/datasets/csv/default-89da93f9a459780b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 4/4 [00:00<00:00, 40.42ba/s]
05/05/2022 11:44:53 - INFO - happytransformer.happy_transformer -   Training...
/Users/yevgeniy_simonov/Documents/Code/Feedmee-app/deep_learning/venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3016
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1131
 44%|████▍     | 500/1131 [13:56<19:19,  1.84s/it]

{'loss': 0.5804, 'learning_rate': 2.7895667550839967e-05, 'epoch': 1.33}


 88%|████████▊ | 1000/1131 [27:59<03:44,  1.71s/it]

{'loss': 0.4375, 'learning_rate': 5.7913351016799295e-06, 'epoch': 2.65}


100%|██████████| 1131/1131 [31:37<00:00,  1.64s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 1131/1131 [31:37<00:00,  1.68s/it]

{'train_runtime': 1897.9385, 'train_samples_per_second': 4.767, 'train_steps_per_second': 0.596, 'train_loss': 0.4986678577966125, 'epoch': 3.0}


In [36]:
# get model loss
before_loss = happy_tt.eval("eval.csv")

print("After loss: ", before_loss.loss)

05/05/2022 12:16:31 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 224.16it/s]


Dataset csv downloaded and prepared to /Users/yevgeniy_simonov/.cache/huggingface/datasets/csv/default-8aed52ffb85829d2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 29.17ba/s]
PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1
100%|██████████| 2988/2988 [05:56<00:00,  8.39it/s]

After loss:  0.448702335357666


In [37]:
from happytransformer import TTSettings

beam_settings = TTSettings(num_beams=5, min_length=1, max_length=20)

In [38]:
# Example 1
string = "grammar: This sentences, has bads grammar and spelling!"
result = happy_tt.generate_text(string, args=beam_settings)
print(result.text)

This sentence has bad grammar and spelling!


In [39]:
# Example 2
string = "grammar: I am enjoys, writtings articles ons AI."
result = happy_tt.generate_text(string, args=beam_settings)
print(result.text)

I enjoy writing articles on AI.


In [40]:
# Example 3
string = "grammar: wht a god day today!"
result = happy_tt.generate_text(string, args=beam_settings)
print(result.text)

Today is a god day!


In [41]:
happy_tt.save("transformer/spelling")

Configuration saved in transformer/spelling/config.json
Model weights saved in transformer/spelling/pytorch_model.bin
tokenizer config file saved in transformer/spelling/tokenizer_config.json
Special tokens file saved in transformer/spelling/special_tokens_map.json
Copy vocab file to transformer/spelling/spiece.model


In [50]:

# Examples
prefix = "grammar: "
strings = [
    "wht a gid day today!",
    "tday is a nace weather.",
    "i lik prgrammg challengs",
    "is has ben windy todya",
    "ban2ns, apples, organges agre all tasty fruits",
    "i went for a wlk and realsed i forgot to turn off kettl",
    "intellgnce is a trate of human hind"
]

for string in strings:
    string_with_prefix = prefix + string
    result = happy_tt.generate_text(string_with_prefix, args=beam_settings)
    print(string, "   ----->   ", result.text)

wht a gid day today!    ----->    What a great day today!
tday is a nace weather.    ----->    Today is a pleasant weather.
i lik prgrammg challengs    ----->    I like programming challengs .
is has ben windy todya    ----->    It has been windy todyas .
ban2ns, apples, organges agre all tasty fruits    ----->    Bananas, apples, organges and all tasty fruits .
i went for a wlk and realsed i forgot to turn off kettl    ----->    I went for a walk and realized I forgot to turn off the lights .
intellgnce is a trate of human hind    ----->    Intelligence is a trait of human hindsight .


In [ ]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
    for rep in replacements:
        text = text.replace(rep[0], rep[1])
    return text

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                if input_text and correction:
                    input_text = remove_excess_spaces(input_text)
                    correction = remove_excess_spaces(correction)
                    writer.writerow([input_text, correction])